# Demo for paper "Liquid Warping GAN: A Unified Framework for Human Motion Imitation, Appearance Transfer and Novel View Synthesis"

### Made just a little bit more accessible by Eyal Gruss (https://eyalgruss.com, eyalgruss@gmail.com)

##### Original project: https://svip-lab.github.io/project/impersonator

##### Original notebook: https://colab.research.google.com/github/svip-lab/impersonator/blob/master/impersonator.ipynb

##### Impersonator++: https://colab.research.google.com/drive/1bwUnj-9NnJA2EMr7eWO4I45UuBtKudg_

#### **Stuff I made**:
##### Avatars4all repository: https://github.com/eyaler/avatars4all
##### Notebook for live webcam in the browser: https://colab.research.google.com/github/eyaler/avatars4all/blob/master/fomm_live.ipynb
##### Notebook for talking head model: https://colab.research.google.com/github/eyaler/avatars4all/blob/master/fomm_bibi.ipynb
##### Notebook for full body models (FOMM): https://colab.research.google.com/github/eyaler/avatars4all/blob/master/fomm_fufu.ipynb
##### Notebook for full body models (impersonator): https://colab.research.google.com/github/eyaler/avatars4all/blob/master/ganozli.ipynb
##### Notebook for full body models (impersonator++): https://colab.research.google.com/github/eyaler/avatars4all/blob/master/ganivut.ipynb
##### Notebook for Wav2Lip audio based lip syncing: https://colab.research.google.com/github/eyaler/avatars4all/blob/master/melaflefon.ipynb
##### List of more generative tools: https://j.mp/generativetools

In [1]:
#@title Setup
%cd /content
!git clone --depth 1 https://github.com/eyaler/impersonator
!pip install DriveDownloader
import os
if not os.path.exists('/content/impersonator/assets/pretrains.zip'):
  !ddl --filename /content/impersonator/assets/pretrains.zip 'https://1drv.ms/u/s!AjjUqiJZsj8whLNw4QyntCMsDKQjSg?e=L77Elv'
if not os.path.exists('/content/impersonator/assets/samples.zip'):
  !ddl --filename /content/impersonator/assets/samples.zip 'https://1drv.ms/u/s!AjjUqiJZsj8whLNz4BqnSgqrVwAXoQ?e=bC86db'
if not os.path.exists('/content/impersonator/outputs/checkpoints.zip'):
  !ddl --filename /content/impersonator/outputs/checkpoints.zip 'https://1drv.ms/u/s!AjjUqiJZsj8whLNyoEh67Uu0LlxquA?e=dkOnhQ'
!pip install -U git+https://github.com/ytdl-org/youtube-dl
!pip install imageio==2.9.0
!pip install imageio-ffmpeg==0.4.5
!pip install torch==1.3.0
!pip install torchvision==0.5.0
!pip install visdom==0.1.8.9
!pip install ipdb==0.13.13
%cd /content/impersonator/thirdparty/neural_renderer
!python setup.py install &> /dev/null
!unzip -qq -u /content/impersonator/assets/pretrains.zip -d /content/impersonator/assets
!unzip -qq -u /content/impersonator/assets/samples.zip -d /content/impersonator/assets
!unzip -qq -u /content/impersonator/outputs/checkpoints.zip -d /content/impersonator/outputs
%cd /content/impersonator

/content
fatal: destination path 'impersonator' already exists and is not an empty directory.
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


  Cloning https://github.com/ytdl-org/youtube-dl to /tmp/pip-req-build-7bej3f8o
  Running command git clone --filter=blob:none --quiet https://github.com/ytdl-org/youtube-dl /tmp/pip-req-build-7bej3f8o
  Resolved https://github.com/ytdl-org/youtube-dl to commit 00ef748cc0e35ee60efd0f7a00e373ab8d1af86b
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement torch==1.3.0 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1)
ERROR: No matching distribution found for torch==1.3.0
ERROR: Could not find a version that satisfies the requirement torchvision==0.5.0 (from versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3, 0.12.0, 0.13.0, 0.13.1, 0.14.0, 0.14.1, 0.15.0, 0.15.1, 0.15.2, 0.16.0, 0.16.1)
ERROR: No matching distribution found for torchvision==0.5.0
/content/impersonator/thirdparty/neural_renderer
/content/impersonator


In [2]:
#@title Get the Driver video and Avatar image from the web
#@markdown 1. You can change the URLs to your **own** stuff!
#@markdown 2. Alternatively, you can upload **local** files in the next cells

video_url = 'https://www.youtube.com/watch?v=NdEPmitJvaA' #@param {type:"string"}
image_url = 'https://i.pinimg.com/564x/74/9e/96/749e96293399ebe1b8c99d89be522383.jpg' #@param {type:"string"}

if video_url:
  !rm -f /content/video.mp4
  !youtube-dl --no-playlist -f "bestvideo[ext=mp4][vcodec!*=av01][height<=360]+bestaudio[ext=m4a]/mp4[height<=360][vcodec!*=av01]/mp4[vcodec!*=av01]/mp4" "$video_url" --merge-output-format mp4 -o /content/video
  !mv /content/video.mp4 /content/video

if image_url:
  !wget "$image_url" -O /content/image

[youtube] NdEPmitJvaA: Downloading webpage
[youtube] Downloading just video NdEPmitJvaA because of --no-playlist
[youtube] NdEPmitJvaA: Downloading player 190c935f
[dashsegments] Total fragments: 1
[download] Destination: /content/video.f134
[download] 100% of 1.00MiB in 00:00
[dashsegments] Total fragments: 1
[download] Destination: /content/video.f140
[download] 100% of 2.01MiB in 00:00
[ffmpeg] Merging formats into "/content/video.mp4"
Deleting original file /content/video.f134 (pass -k to keep)
Deleting original file /content/video.f140 (pass -k to keep)
--2023-11-20 10:57:22--  https://i.pinimg.com/564x/74/9e/96/749e96293399ebe1b8c99d89be522383.jpg
Resolving i.pinimg.com (i.pinimg.com)... 146.75.80.84, 2a04:4e42:84::84
Connecting to i.pinimg.com (i.pinimg.com)|146.75.80.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20147 (20K) [image/jpeg]
Saving to: ‘/content/image’

/content/image      100%[===================>]  19.67K  --.-KB/s    in 0.002s  

20

In [8]:
#@title Optionally upload local Driver video { run: "auto" }
manually_upload_video = True #@param {type:"boolean"}
if manually_upload_video:
  from google.colab import files
  import shutil

  %cd /content/sample_data
  try:
    uploaded = files.upload()
  except Exception as e:
    %cd /content
    raise e

  for fn in uploaded:
    shutil.move('/content/sample_data/'+fn, '/content/video')
    break
  %cd /content

/content/sample_data


Saving supershy_shorts_28s.mp4 to supershy_shorts_28s.mp4
/content


In [9]:
#@title Optionally upload local Avatar image { run: "auto" }
manually_upload_image = True #@param {type:"boolean"}
if manually_upload_image:
  from google.colab import files
  import shutil

  %cd /content/sample_data
  try:
    uploaded = files.upload()
  except Exception as e:
    %cd /content
    raise e

  for fn in uploaded:
    shutil.move('/content/sample_data/'+fn, '/content/image')
    break
  %cd /content

/content/sample_data


Saving hyoeun.jpg to hyoeun.jpg
/content


In [10]:
#@title Optionally shorten Driver video
start_seconds = 0 #@param {type:"number"}
duration_seconds =  20#@param {type:"number"}
start_seconds = max(start_seconds,0)
duration_seconds = max(duration_seconds,0)

if duration_seconds:
  !mv /content/video /content/full_video
  !ffmpeg -ss $start_seconds -t $duration_seconds -i /content/full_video -f mp4 /content/video -y

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [11]:
#@title Prepare assets
#@markdown If you ran out of RAM this means that the video is too large. You can shorten it above.

center_video_to_body = False #@param {type:"boolean"}
crop_video_to_body = False #@param {type:"boolean"}
video_crop_expansion_factor = 1.05 #@param {type:"number"}
center_image_to_body = True #@param {type:"boolean"}
crop_image_to_body = False #@param {type:"boolean"}
image_crop_expansion_factor = 1.05 #@param {type:"number"}
video_crop_expansion_factor = max(video_crop_expansion_factor, 1)
image_crop_expansion_factor = max(image_crop_expansion_factor, 1)

import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML, clear_output
import cv2
import shutil
import os
import warnings
warnings.filterwarnings("ignore")

hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

def fix_dims(im):
    if im.ndim == 2:
        im = np.tile(im[..., None], [1, 1, 3])
    return im[...,:3]

def get_crop(im, center_body=True, crop_body=True, expansion_factor=1, rects=None):
    im = fix_dims(im)
    if (center_body or crop_body) and rects is None:
        rects, _ = hog.detectMultiScale(im, winStride=(4, 4),padding=(8,8), scale=expansion_factor)
    if (center_body or crop_body) and rects is not None and len(rects):
        x0,y0,w,h = sorted(rects, key=lambda x: x[2]*x[3])[-1]
        if crop_body:
            x0 += w//2-h//2
            x1 = x0+h
            y1 = y0+h
        else:
            img_h,img_w = im.shape[:2]
            x0 += (w-img_h)//2
            x1 = x0+img_h
            y0 = 0
            y1 = img_h
    else:
        h,w = im.shape[:2]
        x0 = (w-h)//2
        x1 = (w+h)//2
        y0 = 0
        y1 = h
    return int(x0),int(x1),int(y0),int(y1)

def pad_crop_resize(im, x0=None, x1=None, y0=None, y1=None, new_h=256, new_w=256):
    im = fix_dims(im)
    h,w = im.shape[:2]
    if x0 is None:
      x0 = 0
    if x1 is None:
      x1 = w
    if y0 is None:
      y0 = 0
    if y1 is None:
      y1 = h
    if x0<0 or x1>w or y0<0 or y1>h:
        im = np.pad(im, pad_width=[(max(-y0,0),max(y1-h,0)),(max(-x0,0),max(x1-w,0)),(0,0)], mode='edge')
    return resize(im[max(y0,0):y1-min(y0,0),max(x0,0):x1-min(x0,0)], (new_h, new_w))


source_image = imageio.imread('/content/image')
source_image = pad_crop_resize(source_image, *get_crop(source_image, center_body=center_image_to_body, crop_body=crop_image_to_body, expansion_factor=image_crop_expansion_factor))
imageio.imwrite('/content/crop.png', (source_image*255).astype(np.uint8))

shutil.rmtree('/content/images', ignore_errors=True)
os.makedirs('/content/images')
with imageio.get_reader('/content/video', format='mp4') as reader:
  fps = reader.get_meta_data()['fps']

  driving_video = []
  rects = None
  try:
      for i,im in enumerate(reader):
          if not crop_video_to_body:
              break
          rects, _ = hog.detectMultiScale(im, winStride=(4, 4),padding=(8,8), scale=video_crop_expansion_factor)
          if rects is not None and len(rects):
              break
      x0,x1,y0,y1 = get_crop(im, center_body=center_video_to_body, crop_body=crop_video_to_body, expansion_factor=video_crop_expansion_factor, rects=rects)
      reader.set_image_index(0)
      for j,im in enumerate(reader):
          driving_video.append(pad_crop_resize(im,x0,x1,y0,y1))
          imageio.imwrite(os.path.join('/content/images','%05d.jpg'%j), (driving_video[-1]*255).astype(np.uint8))
  except RuntimeError:
      pass

def vid_display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

clear_output()
if rects is not None and len(rects):
    print('first found body in frame %d'%i)
print('number of frames: %d'%j)
HTML(vid_display(source_image, driving_video).to_html5_video())

number of frames: 599


In [12]:
#@title Animate

bg_ks = 13 #@param {type:"slider", min:1, max:25, step:2}
ft_ks = 3 #@param {type:"slider", min:1, max:25, step:2}
has_detector = True #@param {type:"boolean"}
post_tune = True #@param {type:"boolean"}
front_warp = True #@param {type:"boolean"}
cam_strategy = 'smooth' #@param ['smooth', 'source', 'copy'] {type:"raw"}

run = 'python run_imitator.py --gpu_ids 0 --model imitator --output_dir ./outputs/results --src_path /content/crop.png --tgt_path /content/images --save_res --bg_ks %d --ft_ks %d %s %s %s --cam_strategy %s'%(
    bg_ks, ft_ks, '--has_detector' if has_detector else '', '--post_tune' if post_tune else '', '--front_warp' if front_warp else '', cam_strategy)

%cd /content/impersonator
shutil.rmtree('./outputs/results', ignore_errors=True)
!{run}
!ffmpeg -f image2 -i /content/impersonator/outputs/results/imitators/gt_%05d.jpg -c:v libx264 -c:a aac -pix_fmt yuv420p -profile:v baseline -movflags +faststart out2.mp4 -y
!ffmpeg -f image2 -i /content/impersonator/outputs/results/imitators/pred_%05d.jpg -c:v libx264 -c:a aac -pix_fmt yuv420p -profile:v baseline -movflags +faststart /content/output.mp4 -y
#!ffmpeg -i out2.mp4 -filter_complex "[0:v] palettegen" palette.png -y
#!ffmpeg -i out2.mp4 -i palette.png -filter_complex "[0:v][1:v] paletteuse" endresult.gif -y
#!ffmpeg -i /content/output.mp4 -filter_complex "[0:v] palettegen" palette2.png -y
#!ffmpeg -i /content/output.mp4 -i palette2.png -filter_complex "[0:v][1:v] paletteuse" endresult2.gif -y
#!ffmpeg -i endresult.gif -i endresult2.gif -filter_complex hstack outputstack2.gif -y
#!ffmpeg -i outputstack2.gif -f mp4 -c:v libx264 -c:a aac -pix_fmt yuv420p -profile:v baseline -movflags +faststart /content/combined.mp4
!ffmpeg -i out2.mp4 -i /content/output.mp4 -filter_complex hstack -pix_fmt yuv420p -profile:v baseline -movflags +faststart /content/combined.mp4 -y
#!ffmpeg -i /content/combined.mp4 -i /content/video -c copy -map 0:v -map 1:a? -pix_fmt yuv420p -profile:v baseline -movflags +faststart /content/final.mp4 -y

from IPython.display import HTML, clear_output
from base64 import b64encode

#clear_output()
with open('/content/combined.mp4', 'rb') as f:
  data_url = "data:video/mp4;base64," + b64encode(f.read()).decode()
display(HTML("""
<video width=600 controls autoplay loop>
      <source src="%s" type="video/mp4">
</video>""" % data_url))

/content/impersonator
------------ Options -------------
T_pose: False
batch_size: 4
bg_ks: 13
bg_model: ./outputs/checkpoints/deepfillv2/net_epoch_50_id_G.pth
bg_replace: False
body_seg: False
cam_strategy: smooth
checkpoints_dir: ./outputs/checkpoints/
cond_nc: 3
data_dir: /p300/datasets/iPER
dataset_mode: iPER
debug: False
do_saturate_mask: False
face_model: assets/pretrains/sphere20a_20171020.pth
front_warp: True
ft_ks: 3
gen_name: impersonator
gpu_ids: 0
has_detector: True
hmr_model: assets/pretrains/hmr_tf2pt.pth
image_size: 256
images_folder: images_HD
ip: 
is_train: False
load_epoch: 0
load_path: ./outputs/checkpoints/lwb_imper_fashion_place/net_epoch_30_id_G.pth
map_name: uv_seg
model: imitator
n_threads_test: 2
name: running
norm_type: instance
only_vis: False
output_dir: ./outputs/results
part_info: assets/pretrains/smpl_part_info.json
port: 31100
post_tune: True
pri_path: ./assets/samples/A_priors/imgs
repeat_num: 6
save_res: True
serial_batches: False
smpl_model: assets/pr

In [ ]:
#@title Download
#@markdown 1. If it fails try running this cell again.
#@markdown 2. Alternatively, you can manually download "output.mp4", "combined.mp4" from the folder on the left (click "Refresh" if missing).

print() #see https://github.com/googlecolab/colabtools/issues/468
from google.colab import files
files.download('/content/combined.mp4') #fails for Firefox private window
files.download('/content/output.mp4')

'''
palette="palette.png"
filters="fps=15,scale=256:-1:flags=lanczos"
!ffmpeg -v warning -i /content/output.mp4 -vf "$filters,palettegen=stats_mode=diff" -y $palette
!ffmpeg -v warning -i /content/output.mp4 -i $palette -lavfi "$filters [x]; [x][1:v] paletteuse=dither=bayer:bayer_scale=5:diff_mode=rectangle" -y /content/output.gif
filters="fps=15,scale=512:-1:flags=lanczos"
!ffmpeg -v warning -i /content/combined.mp4 -vf "$filters,palettegen=stats_mode=diff" -y $palette
!ffmpeg -v warning -i /content/combined.mp4 -i $palette -lavfi "$filters [x]; [x][1:v] paletteuse=dither=bayer:bayer_scale=5:diff_mode=rectangle" -y /content/combined.gif
files.download('/content/combined.gif')
files.download('/content/output.gif')
'''
pass